In [1]:
# %load_ext autoreload
# %autoreload 2
import models, utils
import orca
import dask.dataframe as dd
from __future__ import division
import csv
import numpy as np
import pandas as pd
import logging
import os
from urbansim.utils import yamlio
from urbansim.utils import misc
#import greenroof


In [21]:
greenroof.NoInterventionGreenRoof()
greenroof.SanFranGreenRoof()
greenroof.ChicagoGreenRoof()
greenroof.TorontoGreenRoof()
print("success")

success


In [17]:
buildings = orca.get_table('buildings')
buildings.to_frame()
print(orca.list_columns())
mybuild = orca.orca.DataFrameWrapper.get_column(buildings, 'unit_sqft')
mybuild.to_frame()
mybuild.sort_index(inplace=True)
mybuild.to_frame()
height = orca.orca.DataFrameWrapper.get_column(buildings, 'stories')
height.sort_index(inplace=True)
height.to_frame()
purpose = orca.orca.DataFrameWrapper.get_column(buildings, 'general_type')
purpose.sort_index(inplace=True)
purpose.to_frame()
dataset = pd.concat([height, mybuild, purpose], axis=1)
dataset
buildings.to_frame()
#dataset.to_csv('data.csv') #uncomment to place is csv file and fix formatting
#height of building approx = 11.5 feet * stories

[('parcels', 'land_cost'), ('zones', 'sum_residential_units'), ('parcels', 'max_far'), ('buildings', 'zone_id'), ('buildings', 'general_type'), ('zones_prices', 'residential'), ('jobs', 'zone_id'), ('zones', 'ave_income'), ('zones', 'poor'), ('buildings', 'vacant_job_spaces'), ('zones_prices', 'industrial'), ('zones_prices', 'retail'), ('buildings', 'unit_lot_size'), ('households', 'zone_id'), ('parcels', 'total_job_spaces'), ('zones_prices', 'zone_id'), ('zones', 'hhsize'), ('parcels', 'max_height'), ('parcels', 'parcel_size'), ('buildings', 'job_spaces'), ('zones', 'zone_id'), ('zones', 'renters'), ('buildings', 'sqft_per_job'), ('buildings', 'unit_sqft'), ('zones', 'sfdu'), ('zones', 'sum_job_spaces'), ('households', 'income_quartile'), ('zones', 'ave_unit_sqft'), ('zones', 'population'), ('buildings', 'vacant_residential_units'), ('zones', 'ave_lot_sqft'), ('parcels', 'total_units'), ('zones', 'jobs'), ('zones_prices', 'office'), ('parcels', 'ave_unit_size'), ('parcels', 'total_sqf

,building_sqft,building_type_id,non_residential_rent,non_residential_sqft,parcel_id,residential_sales_price,residential_units,stories,year_built,vacant_job_spaces,job_spaces,sqft_per_job,unit_lot_size,vacant_residential_units,unit_sqft,general_type,zone_id
building_id,,,,,,,,,,,,,,,,,
66515,4755.000000,3.0,0,0.0,69377,304,3.0,3.0,1900,0.0,0,400,1263.125000,0.0,1585.000000,Residential,103.0
123898,3770.000000,3.0,0,0.0,131763,202,2.0,2.0,1923,0.0,0,400,1683.608276,0.0,1885.000000,Residential,60.0
44682,2275.000000,2.0,0,0.0,46013,106,1.0,1.0,1931,0.0,0,400,4439.019043,0.0,2275.000000,Residential,134.0
62974,4032.000000,3.0,0,0.0,65342,361,3.0,3.0,1910,0.0,0,400,919.461670,0.0,1344.000000,Residential,82.0
55586,1225.000000,2.0,0,0.0,57438,391,1.0,1.0,1927,0.0,0,400,2498.522217,0.0,1225.000000,Residential,168.0
78144,1500.000000,2.0,0,0.0,83380,181,1.0,1.0,1998,0.0,0,400,2512.025391,-1.0,1500.000000,Residential,143.0
17824,1747.000000,2.0,0,0.0,18432,232,1.0,1.0,1930,0.0,0,400,2998.202881,-1.0,1747.000000,Residential,184.0
130468,1435.000000,3.0,0,0.0,139244,338,2.0,2.0,1908,0.0,0,400,1061.887573,0.0,717.500000,Residential,19.0
23147,1137.000000,2.0,0,0.0,23886,435,1.0,1.0,1948,0.0,0,400,2380.722412,-1.0,1137.000000,Residential,183.0


In [89]:
buildings = orca.get_table('buildings')
buildings.to_frame()
buildings['general_type'][1]

'Residential'

In [46]:
#TO DO: Gradually Transfer this alg to data cell below

data = pd.read_csv('data.csv')
roofinstalled_dict = {}
roof_cost_at_20yrs_dict = {}
green_roof_benefit_dict = {}
green_roof_cost_dict = {}
green_roof_net_benefit_dict = {}     
building_size_dict = {}
#greenroof.NoInterventionGreenRoof()
green_cost_per_sqft = 12

for index in range(-1, 1000, 1):
    index = index + 1
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    building_size_dict.update({index : building_size})
    #Initialization of assumed and derived variables
    base_green_cost_per_sqft = 12
    green_roof_min = 0
    conventional_cost_per_sqft = green_cost_per_sqft/2
    green_roof_benefits = 0
    building_age = 40
    annual_benefit_per_area = .23
    annual_maintenance_costs_per_area = 0.02 
    min_green_area = green_roof_min * building_size
    min_green_cost = green_roof_min * green_cost_per_sqft
    best_green_cost = min_green_cost #default value
    min_green_benefits= green_roof_min*building_size*annual_benefit_per_area*building_age
    best_green_net_benefit = min_green_benefits #default value
    best_size = green_roof_min
    best_benefit = min_green_benefits - min_green_cost
    best_conv = (building_size*(1-green_roof_min)*conventional_cost_per_sqft) #Avoid capital costs
    roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final



    #Calculation of Output variables

    #If the building is large enough to allow bulk discounting to begin (happens in most cases)
    if (building_size > 500):
        green_cost_per_sqft = 23.907*(base_green_cost_per_sqft**(-0.078))

    #for current_size in my_range(green_roof_min, 1, 0.1):
    #    roof_cost_at_20yrs = (building_size*(1-current_size)*conventional_cost_per_sqft) #Avoid capital costs
    #    green_roof_cost = current_size * building_size * green_cost_per_sqft + (building_age * annual_maintenance_costs_per_area * building_size * current_size)
    #    green_roof_benefits = building_age * annual_benefit_per_area * building_size * current_size + roof_cost_at_20yrs
    #    if(green_roof_benefits - green_roof_cost > best_benefit):
    #        best_size = current_size
    #        best_green_cost = green_roof_cost
    #        best_green_benefits = green_roof_benefits
    #        best__green_net_benefit = green_roof_benefits - green_roof_cost
    #        best_conv = roof_cost_at_20yrs

    if(best_benefit > 0):
        roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final
    else:
        roofinstalled_dict.update({index : 0})

    roof_cost_at_20yrs_dict.update({index : best_conv})
    green_roof_benefit_dict.update({index : best_benefit})
    green_roof_cost_dict.update({index : best_green_cost})
    green_roof_net_benefit_dict.update({index : best_green_net_benefit})
    index = index +1   

In [96]:
data['building_type_id'][2]

'Residential'

In [66]:
#WORK SHOULD BE DONE IN HERE
data = pd.read_csv('data.csv')
def my_range(start, end, step):
    while start <= end:
        yield start #keyword yield generates an interator
        start += step

building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}

for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    
    for size_ratio in my_range (0, 1.01, .01):
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        if(green_roof_ben - green_roof_cost > best_ben):
            roof_ins = 1
            best_size = size_ratio * building_size
            size_dict.update({index : best_size})
            roofins_dict.update({index : roof_ins})
            green_roof_ben_dict.update({index : green_roof_ben})
            green_roof_cost_dict.update({index : green_roof_cost})
            roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
            roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, })
writer = pd.ExcelWriter('no_action.xlsx')
buildings.to_excel(writer,'no_action')
writer.save()

In [67]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    for size_ratio in my_range (0, 1.01, .01):
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio)) + 0.05*building_size*size_ratio
        if(green_roof_ben - green_roof_cost > best_ben):
            roof_ins = 1
            best_size = size_ratio * building_size
            size_dict.update({index : best_size})
            roofins_dict.update({index : roof_ins})
            green_roof_ben_dict.update({index : green_roof_ben})
            green_roof_cost_dict.update({index : green_roof_cost})
            roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
            roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict})
writer = pd.ExcelWriter('chicago.xlsx')
buildings.to_excel(writer,'chicago')
writer.save()

In [68]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
mandated_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    
    if(data['building_type_id'][index] != 'Residential' and building_size < 15625):
        mandated_dict.update({index : 1})
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        roof_ins = 1
        best_size = building_size
        size_dict.update({index : building_size})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})
        
    elif(data['building_type_id'][index] != 'Residential' and building_size >= 15625):
        mandated_dict.update({index : 1})
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
        green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
        roof_ins = 1
        best_size = 15625
        size_dict.update({index : 15625})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})
    
    else:
        mandated_dict.update({index : 0})
        for size_ratio in my_range (0, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, "mandated" : mandated_dict})
writer = pd.ExcelWriter('san_fran.xlsx')
buildings.to_excel(writer,'san_fran')
writer.save()

In [69]:
building_size_dict = {}
roofins_dict = {}
green_roof_ben_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
mandated_dict = {}
green_roof_min_dict = {}
for index in range(-1, 2000, 1):
    index = index + 1
    best_ben = 0
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    roof_cost_at_20yrs = building_size * 6
    building_size_dict.update({index : building_size})
    #REVERT OFFICE #51 to 1330 sq.ft
    if(data['building_type_id'][index] != 'Residential' or ((data['building_type_id'][index] == 'Residential' and data['stories'][index] > 6))):
        mandated_dict.update({index : 1})
        if(building_size > 21525 and building_size <= 53800):
            green_roof_min = .2
        elif(building_size > 53800 and building_size <= 107625):
            green_roof_min = .3 
        elif(building_size > 107625 and building_size <= 161450):
            green_roof_min = .4 
        elif(building_size > 161450 and building_size <= 215275):
            green_roof_min = .5 
        elif (building_size > 215275):
            green_roof_min = .6
        else:
            green_roof_min = 0
              
        green_roof_min_dict.update( {index : green_roof_min})
        green_roof_cost = 23.907*(12**(-0.078)) * building_size + (40*.21*building_size*green_roof_min)
        green_roof_ben = 1.23 * building_size * 40 + (building_size * 6 * (1-green_roof_min))
        roof_ins = 1
        best_size = building_size * green_roof_min
        size_dict.update({index : best_size})
        roofins_dict.update({index : roof_ins})
        green_roof_ben_dict.update({index : green_roof_ben})
        green_roof_cost_dict.update({index : green_roof_cost})
        roof_cost_at_20yrs_dict.update({index : 0})

        for size_ratio in my_range (green_roof_min, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

    else:
        mandated_dict.update({index : 0})
        green_roof_min_dict.update( {index : 0})
        for size_ratio in my_range (0, 1.01, .01):
            green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size + (40*.21*building_size*size_ratio)
            green_roof_ben = .23 * size_ratio * building_size * 40 + (building_size * 6 * (1-size_ratio))
            if(green_roof_ben - green_roof_cost > best_ben):
                roof_ins = 1
                best_size = size_ratio * building_size
                size_dict.update({index : best_size})
                roofins_dict.update({index : roof_ins})
                green_roof_ben_dict.update({index : green_roof_ben})
                green_roof_cost_dict.update({index : green_roof_cost})
                roof_cost_at_20yrs = building_size * 6 * (1-size_ratio)            
                roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})

buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_ben" : green_roof_ben_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict, "mandated" : mandated_dict, "green_min" : green_roof_min_dict})
writer = pd.ExcelWriter('toronto.xlsx')
buildings.to_excel(writer,'toronto')
writer.save()


In [ ]:
parcels = orca.get_table('parcels')
parcels.to_frame()

In [ ]:
buildings = orca.get_table('buildings')
buildings.to_frame()

In [ ]:
zones = orca.get_table('zones')
zones.to_frame()

In [1]:
import models, utils
import orca
import dask.dataframe as dd
from __future__ import division
import csv
import numpy as np
import pandas as pd
import logging
import os
from urbansim.utils import yamlio
from urbansim.utils import misc

#FROM URBANSIM2 https://github.com/psrc/urbansim2/blob/master/simulate.py#L29
@orca.injectable('simfile')
def simfile():
     return "simresult29Mar18.h5"

# remove results file if exists
outfile = simfile()
if os.path.exists(outfile):
     os.remove(outfile)
     

    
orca.run([
    "rsh_simulate",              # residential sales hedonic
    "nrh_simulate",              # non-residential rent hedonic

    "households_relocation",     # households relocation model
    "hlcm_simulate",            # households location choice
    "households_transition",     # households transition

    "jobs_relocation",           # jobs relocation model
    "elcm_simulate",             # employment location choice
    "jobs_transition",           # jobs transition

    "feasibility",               # compute development feasibility
    "residential_developer",     # build residential buildings
    "non_residential_developer", # build non-residential buildings
], iter_vars=[2010, 2011, 2012, 2013, 2014, 2015, 2016], data_out=outfile, compress=True, out_run_local=True) 
logging.info('Simulation finished')
print('Simulation finished')

Filling column job_category with value service (331 values)
Filling column residential_sales_price with value 0 (14196 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column residential_units with value 0 (0 values)
Filling column building_type_id with value 2.0 (0 values)
Running iteration 1 with iteration value 2010
Running step 'rsh_simulate'
count    1.407800e+05
mean     4.271533e+03
std      3.744239e+05
min      1.204550e-04
25%      1.899947e+02
50%      3.179589e+02
75%      5.150183e+02
max      1.210341e+08
dtype: float64
Time to execute step 'rsh_simulate': 0.99 s
Running step 'nrh_simulate'
count    10803.000000
mean        20.729909
std          6.833292
min          6.258496
25%         17.431477
50%         20.908725
75%         25.077906
max         47.266957
dtype: float64
Time to execute step 'nrh_simulate': 0.74 s


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:394: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_industrial_height] = np.nan
C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_retail_height] = np.nan


Describe of the yearly rent by use
              retail    industrial         office    residential
count  152205.000000  94493.000000  152259.000000  153228.000000
mean       23.867557      8.650493      21.274562      16.511098
std         3.553082      1.641869       2.979941       5.507942
min        18.000000      6.000000      16.000000       0.500000
25%              NaN           NaN            NaN            NaN
50%              NaN           NaN            NaN            NaN
75%              NaN           NaN            NaN            NaN
max        35.000000     14.500000      31.000000      72.800000
Computing feasibility for form mixedoffice


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Computing feasibility for form industrial
Computing feasibility for form retail
Computing feasibility for form mixedresidential
Computing feasibility for form office
Computing feasibility for form residential
Time to execute step 'feasibility': 4.63 s
Running step 'residential_developer'
Number of agents: 362,867
Number of agent spaces: 374,133
Current vacancy = 0.03
Target vacancy = 0.15, target of new units = 52,769
3,838 feasible buildings before running developer
Sum of net units that are profitable: 33,312
WARNING THERE WERE NOT ENOUGH PROFITABLE UNITS TO MATCH DEMAND
Adding 1,695 buildings with 38,694 residential_units
2,143 feasible buildings after running developer
Time to execute step 'residential_developer': 0.71 s
Running step 'non_residential_developer'
Number of agents: 236,711
Number of agent spaces: 642,004
Current vacancy = 0.63
Target vacancy = 0.15, target of new units = 0
2,143 feasible buildings before running developer
Sum of net units that are profitable: 108,914


In [11]:
import h5py
filename = 'simresult29Mar18.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])
data
a_group_key

Keys: KeysView(<HDF5 file "simresult29Mar18.h5" (mode r)>)


'2010'

In [17]:
import pandas as pd
data = pd.read_hdf('simresult29Mar18.h5')

HDF5ExtError: HDF5 error back trace

  File "C:\aroot\work\hdf5-1.8.15-patch1\src\H5F.c", line 604, in H5Fopen
    unable to open file
  File "C:\aroot\work\hdf5-1.8.15-patch1\src\H5Fint.c", line 1026, in H5F_open
    file is already open for read-only

End of HDF5 error back trace

Unable to open/create file 'simresult29Mar18.h5'

In [1]:
%load_ext autoreload
%autoreload 2
import models
import orca
from urbansim.maps import dframe_explorer

In [6]:
d = {tbl: orca.get_table().to_frame(tbl) for tbl in ['buildings', 'jobs', 'households', 'zones', 'zones_prices']}

Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column building_type_id with value 2.0 (0 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column residential_sales_price with value 0 (14196 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column job_category with value service (331 values)


In [ ]:
dframe_explorer.start(d, 
        center=[37.7792, -122.2191],
        zoom=11,
        shape_json='data/zones.json',
        geom_name='ZONE_ID', # from JSON file
        join_name='zone_id', # from data frames
        precision=2)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://localhost:8765/
Hit Ctrl-C to quit.

C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py:60: ResourceWarning: unclosed file <_io.TextIOWrapper name='C:\\Users\\Harrison Freund\\Anaconda3\\lib\\site-packages\\urbansim\\maps\\dframe_explorer.html' mode='r' encoding='cp1252'>
  index = open(os.path.join(dir, 'dframe_explorer.html')).read()
127.0.0.1 - - [05/Apr/2018 14:03:19] "GET / HTTP/1.1" 200 28075
127.0.0.1 - - [05/Apr/2018 14:03:21] "GET /data/zones.json HTTP/1.1" 200 7389418
127.0.0.1 - - [05/Apr/2018 14:03:21] "GET /favicon.ico HTTP/1.1" 404 742


df.groupby('zone_id')['parcel_id'].mean()


Traceback (most recent call last):
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in map_query
    results = {int(k): results[k] for k in results}
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in <dictcomp>
    results = {int(k): results[k] for k in results}
ValueError: invalid literal for int() with base 10: '181.0'
127.0.0.1 - - [05/Apr/2018 14:03:28] "GET /map_query/buildings/empty/zone_id/parcel_id/mean() HTTP/1.1" 500 1736


df.groupby('zone_id')['stories'].mean()


Traceback (most recent call last):
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in map_query
    results = {int(k): results[k] for k in results}
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in <dictcomp>
    results = {int(k): results[k] for k in results}
ValueError: invalid literal for int() with base 10: '181.0'
127.0.0.1 - - [05/Apr/2018 14:03:46] "GET /map_query/buildings/empty/zone_id/stories/mean() HTTP/1.1" 500 1734


...

127.0.0.1 - - [05/Apr/2018 14:05:40] "GET /map_query/zones/empty/zone_id/jobs/mean() HTTP/1.1" 200 4832


df.groupby('zone_id')['parcel_id'].mean()


Traceback (most recent call last):
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in map_query
    results = {int(k): results[k] for k in results}
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in <dictcomp>
    results = {int(k): results[k] for k in results}
ValueError: invalid literal for int() with base 10: '181.0'
127.0.0.1 - - [05/Apr/2018 14:05:55] "GET /map_query/buildings/empty/zone_id/parcel_id/mean() HTTP/1.1" 500 1736


df.groupby('zone_id')['non_residential_sqft'].mean()


Traceback (most recent call last):
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in map_query
    results = {int(k): results[k] for k in results}
  File "C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\maps\dframe_explorer.py", line 47, in <dictcomp>
    results = {int(k): results[k] for k in results}
ValueError: invalid literal for int() with base 10: '181.0'
127.0.0.1 - - [05/Apr/2018 14:06:01] "GET /map_query/buildings/empty/zone_id/non_residential_sqft/mean() HTTP/1.1" 500 1747


In [77]:
import numpy
a = [1, 1, 1 ,1, 1]
ar = numpy.array(a)
ar = ar+2
print (ar + 2)

[5 5 5 5 5]
